In [3]:
from operator import itemgetter
import pandas as pd

def search_similar_movies_2(search_term):
    query_bow = dictionary.doc2bow(tokenizer(search_term))
    query_tfidf = movie_tfidf_model[query_bow]
    query_lsi = movie_lsi_model[query_tfidf]
    
    movies_list = movie_index[query_lsi]
    movies_list.sort(key=itemgetter(1), reverse=True)
    
    results_per_page = 10
    start_index = 1
    
    # Function to truncate the plot to first 20 words
    def truncate_plot(plot_text):
        words = plot_text.split()
        if len(words) <= 10:
            return plot_text
        return ' '.join(words[:10]) + '...'
    
    while start_index < len(movies_list):
        movie_names = []
        
        # Store original movie data for later full plot display
        current_page_movies = []
        
        for j, movie in enumerate(movies_list[start_index:start_index + results_per_page]):
            movie_index_in_df = movie[0]
            movie_title = df_movies['Title'][movie_index_in_df]
            movie_plot = df_movies['Plot'][movie_index_in_df]
            
            movie_names.append({
                'Index': j,  # Add display index for selection
                'Relevance': round((movie[1] * 100), 2),
                'Movie Title': movie_title,
                'Movie Plot': truncate_plot(movie_plot)
            })
            
            # Store complete information
            current_page_movies.append({
                'Index': movie_index_in_df,
                'Title': movie_title,
                'Plot': movie_plot,
                'Relevance': round((movie[1] * 100), 2)
            })
        
        results_df = pd.DataFrame(movie_names, columns=['Index', 'Relevance', 'Movie Title', 'Movie Plot'])
        
        # Set display options
        pd.set_option('display.max_colwidth', None)  # Don't truncate column contents
        pd.set_option('display.expand_frame_repr', False)  # Try to show all columns in one row
        pd.set_option('display.max_rows', 10)  # Show only 10 rows max
        pd.set_option('display.precision', 2)  # Show 2 decimal places for floats
        pd.set_option('display.colheader_justify', 'left')  # Left-align column headers

        
        
        print("\n" + "="*80)
        print(f"SEARCH RESULTS FOR: '{search_term}' (Page {start_index//results_per_page + 1})")
        print("="*80)

        styled_df = results_df.style.background_gradient(
        subset=['Relevance'], cmap='Blues', high=0.5
        ).set_properties(**{'text-align': 'left'})
        display(styled_df)
        
        # Ask user if they found the movie or want to see full plot
        user_input = input("\nDid you find the movie you were looking for? (yes/no) or enter a movie index number to see full plot: ").strip().lower()
        
        # Check if user entered a number
        try:
            movie_idx = int(user_input)
            if 0 <= movie_idx < len(current_page_movies):
                # Display full details for the selected movie
                selected_movie = current_page_movies[movie_idx]
                print("\n" + "="*80)
                print(f"FULL DETAILS FOR: {selected_movie['Title']}")
                print(f"Relevance Score: {selected_movie['Relevance']}%")
                print("="*80)
                print("\nPLOT SUMMARY:")
                print(selected_movie['Plot'])
                print("\n" + "="*80)
                
                # Ask if they want to continue searching or return this movie
                continue_input = input("\nIs this the movie you were looking for? (yes/no): ").strip().lower()
                if continue_input == 'yes':
                   return pd.DataFrame([{
                        'Index': selected_movie['Index'],
                        'Relevance': "{:.2f}%".format(selected_movie['Relevance']),
                        'Title': selected_movie['Title'],
                        'Plot': selected_movie['Plot']
                        }], columns=['Index', 'Relevance', 'Title', 'Plot'])
                # If no, continue with the loop and show the same page again
                continue
            else:
                print("Invalid movie index. Please try again.")
                continue
        except ValueError:
            # Not a number, process as yes/no
            pass
        
        if user_input == 'yes':
            # Ask which movie they want to return
            while True:
                try:
                    selection = int(input("Enter the index of the movie you want to select: "))
                    if 0 <= selection < len(current_page_movies):
                        current_page_movies[selection]['Relevance'] = f"{current_page_movies[selection]['Relevance']:.2f}%"
                        return pd.DataFrame([current_page_movies[selection]],columns=['Index', 'Relevance', 'Title', 'Plot'])
                    else:
                        print("Invalid index. Please try again.")
                except ValueError:
                    print("Please enter a valid number.")
        elif user_input == 'no':
            #start_index += results_per_page ( if we want to see the next 10 only)
            results_per_page += 5
        else:
            print("Invalid input. Please enter 'yes', 'no', or a movie index.")
    
    print("No more results found.")
    return None

In [ ]:
search_similar_movies_2('love affair hate')